In [ ]:
import re
import requests
import pandas as pd
from io import StringIO
import os

URL = "https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state"
TARGET_YEARS = ["2012", "2016", "2020", "2024"]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

FIFTY_STATES = {
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida",
    "Georgia","Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine",
    "Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska",
    "Nevada","New Hampshire","New Jersey","New Mexico","New York","North Carolina","North Dakota",
    "Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island","South Carolina","South Dakota",
    "Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"
}

def normalize_party_value(val):
    if pd.isna(val): return None
    txt = str(val).strip()
    if txt == "D": return "Democratic"
    if txt == "R": return "Republican"
    if re.search(r"Democrat", txt, re.I): return "Democratic"
    if re.search(r"Republican", txt, re.I): return "Republican"
    if re.fullmatch(r"[dD]", txt): return "Democratic"
    if re.fullmatch(r"[rR]", txt): return "Republican"
    return txt

def normalize_state(s):
    if s is None: return ""
    return re.sub(r"\[[^\]]+\]", "", str(s)).strip()

html = requests.get(URL, headers=HEADERS).text
tables = pd.read_html(StringIO(html), flavor="lxml", header=0)

table = None
for t in tables:
    cols = " ".join(map(str, t.columns))
    if "2012" in cols and "2024" in cols:
        table = t
        break

if table is not None:
    table.columns = [str(c).strip() for c in table.columns]
    state_col = next((c for c in table.columns if "State" in c or "state" in c), table.columns[0])
    sub = table[[state_col] + [c for c in table.columns if any(y in str(c) for y in TARGET_YEARS)]].copy()
    sub.columns = ["State"] + TARGET_YEARS
    sub["State"] = sub["State"].map(normalize_state)
    sub = sub[sub["State"].isin(FIFTY_STATES)].copy()
    for y in TARGET_YEARS:
        sub[y] = sub[y].map(normalize_party_value)
    sub = sub.sort_values("State").reset_index(drop=True)

    duplicate_map = {
        "2013": "2012", "2014": "2012", "2015": "2012",
        "2017": "2016", "2018": "2016", "2019": "2016",
        "2021": "2020", "2022": "2020", "2023": "2020",
        "2025": "2024",
    }
    for new_year, base_year in duplicate_map.items():
        sub[new_year] = sub[base_year]

    FULL_YEARS = [str(y) for y in range(2012, 2026)]
    sub = sub[["State"] + FULL_YEARS]

    def tendency(row):
        reps = sum(row[y] == "Republican" for y in TARGET_YEARS)
        dems = sum(row[y] == "Democratic" for y in TARGET_YEARS)
        if reps > dems:  return "Republican"
        if dems > reps:  return "Democratic"
        return "Tie"

    sub["Tendency"] = sub.apply(tendency, axis=1)

    TOTAL_STATES = len(sub)
    winners_row = {"State": "Winner by year", "Tendency": ""}
    perc_row    = {"State": "Percentages",   "Tendency": ""}

    for y in FULL_YEARS:
        r = (sub[y] == "Republican").sum()
        d = (sub[y] == "Democratic").sum()
        winners_row[y] = "Republican" if r > d else ("Democratic" if d > r else "Tie")
        perc_row[y]    = f"R:{r/TOTAL_STATES*100:.1f}% / D:{d/TOTAL_STATES*100:.1f}%"

    blank_row = {col: "" for col in sub.columns}
    final = pd.concat([sub, pd.DataFrame([blank_row, winners_row, perc_row])], ignore_index=True)

    us_presidential_results = final.copy()

    pd.set_option("display.width", None)
    pd.set_option("display.max_columns", None)
    print(us_presidential_results.head(10))

In [ ]:
import pandas as pd
import numpy as np
import re

try:
    us_presidential_results
except NameError:
    raise RuntimeError(
        "DataFrame 'us_presidential_results' do not exist"
    )

df = pd.read_csv(r"C:\Users\river\Downloads\gva_mass_shootings-2025-10-21.csv")

def norm_state(s):
    if pd.isna(s):
        return ""
    s = re.sub(r"\[[^\]]+\]", "", str(s)).strip()
    s = s.replace("Washington, D.C.", "Washington").replace("D.C.", "Washington")
    if s == "District of Columbia":
        s = "Washington"
    return s

year_cols = [c for c in us_presidential_results.columns if str(c).isdigit() and 2012 <= int(c) <= 2025]

elections_long = (
    us_presidential_results
    .loc[:, ["State"] + year_cols]
    .melt(id_vars=["State"], value_vars=year_cols,
          var_name="Year", value_name="PresidentialResult")
    .dropna(subset=["PresidentialResult"])
)
elections_long["Year"] = elections_long["Year"].astype(int)
elections_long["_state_key"] = elections_long["State"].map(norm_state).str.lower()

pres_map = (
    elections_long
    .set_index(["_state_key", "Year"])["PresidentialResult"]
    .to_dict()
)

state_col = next((c for c in df.columns if "state" in c.lower()), None)
if state_col is None:
    raise ValueError("No found column in CSV.")

year_col = next((c for c in df.columns if c.lower() == "year"), None)
if year_col is None:
    date_col = next((c for c in df.columns if "date" in c.lower()), None)
    if date_col is None:
        raise ValueError("No year no date.")
    year_aux = pd.to_datetime(df[date_col], errors="coerce").dt.year.astype("Int64")
else:
    year_aux = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")

_state_key = df[state_col].map(norm_state).str.lower()
_year_key  = year_aux

df["PresidentialResult"] = [
    pres_map.get((s, int(y)) if pd.notna(y) else (s, y), np.nan)
    for s, y in zip(_state_key, _year_key)
]

df_merged = df.copy()

cols_to_show = [state_col]
if year_col:
    cols_to_show.append(year_col)
cols_to_show.append("PresidentialResult")
print(df_merged[cols_to_show].head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

state_col = next((c for c in df_merged.columns if "state" in c.lower()), None)
if state_col is None:
    raise ValueError("Non value")

if "year" not in df_merged.columns and "Year" in df_merged.columns:
    df_merged = df_merged.rename(columns={"Year": "year"})

if "year" not in df_merged.columns:
    raise ValueError("Non value")

df_plot = df_merged.copy()
df_plot["year"] = pd.to_numeric(df_plot["year"], errors="coerce")
df_plot = df_plot.dropna(subset=["year"])
df_plot["year"] = df_plot["year"].astype(int)

incident_counts = (
    df_plot.groupby(["year", state_col, "PresidentialResult"])
           .size()
           .reset_index(name="Incidents")
)

party_palette = {
    "Democratic": "#1f77b4",   
    "Republican": "#d62728",  
    "Tie": "gray"
}

years = sorted(incident_counts["year"].unique())

fig, axes = plt.subplots(len(years), 1, figsize=(10, 4 * len(years)), sharex=True)

if len(years) == 1:
    axes = [axes]

for i, yr in enumerate(years):
    subset = (
        incident_counts[incident_counts["year"] == yr]
        .nlargest(5, "Incidents")
    )
    sns.barplot(
        data=subset,
        x="Incidents",
        y=state_col,
        hue="PresidentialResult",
        ax=axes[i],
        dodge=False,
        palette=party_palette
    )
    axes[i].set_title(f"Top 5 states with more incidents in {yr}")
    axes[i].set_xlabel("Number of incidents")
    axes[i].set_ylabel("State")
    axes[i].legend(title="ELECTION RESULTS", loc="best")

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

state_totals = (
    df_merged.groupby(["state", "PresidentialResult"])
             .size()
             .reset_index(name="TotalIncidents")
)

top10 = state_totals.nlargest(10, "TotalIncidents")

party_palette = {
    "Democratic": "#1f77b4",   # azul
    "Republican": "#d62728",   # rojo
    "Tie": "gray"
}

plt.figure(figsize=(10, 6))
sns.barplot(
    data=top10,
    x="TotalIncidents",
    y="state",
    hue="PresidentialResult",
    dodge=False,
    palette=party_palette
)
plt.title("Top 10 States with more incidents (2012–2025)", fontsize=14)
plt.xlabel("Total of incidents reported")
plt.ylabel("State")
plt.legend(title="Political Tendency")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.express as px
import re
import numpy as np

state_col = next((c for c in df_merged.columns if "state" in c.lower()), None)
if state_col is None:
    raise ValueError("No column state foun in df_merged.")

if "year" not in df_merged.columns and "Year" in df_merged.columns:
    df_merged = df_merged.rename(columns={"Year": "year"})

def norm_state(s):
    if pd.isna(s):
        return ""
    s = re.sub(r"\[[^\]]+\]", "", str(s)).strip()
    s = s.replace("Washington, D.C.", "Washington").replace("D.C.", "Washington")
    if s == "District of Columbia":
        s = "Washington"
    return s

tmp = df_merged.copy()
tmp["_StateNorm"] = tmp[state_col].map(norm_state)

state_totals = (
    tmp.groupby(["_StateNorm", "PresidentialResult"])
       .size()
       .reset_index(name="TotalIncidents")
)

predominant = (
    state_totals.sort_values(["_StateNorm", "TotalIncidents"], ascending=[True, False])
                .drop_duplicates(subset=["_StateNorm"])
                .rename(columns={"PresidentialResult": "Party"})
)

STATE_TO_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO",
    "Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID",
    "Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY","Louisiana":"LA",
    "Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN",
    "Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV",
    "New Hampshire":"NH","New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC",
    "North Dakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA",
    "Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD","Tennessee":"TN","Texas":"TX",
    "Utah":"UT","Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV",
    "Wisconsin":"WI","Wyoming":"WY"
}

predominant["state_code"] = predominant["_StateNorm"].map(STATE_TO_ABBR)

party_colors = {
    "Democratic": "blue",
    "Republican": "red",
    "Tie": "gray"
}

map_df = predominant.dropna(subset=["state_code"]).copy()

fig = px.choropleth(
    map_df,
    locations="state_code",
    locationmode="USA-states",
    color="Party",
    hover_name="_StateNorm",
    hover_data={"TotalIncidents": True, "state_code": False, "Party": True},
    color_discrete_map=party_colors,
    scope="usa",
    title="Total Incidents 2012–2025 per State and Predominant Party"
)
fig.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    title_x=0.5
)
fig.show()

In [ ]:
line_data = (
    df_merged.groupby(["year", "PresidentialResult"])
    .size()
    .reset_index(name="Incidents")
)

plt.figure(figsize=(10,6))
sns.lineplot(
    data=line_data,
    x="year",
    y="Incidents",
    hue="PresidentialResult",
    palette={"Democratic": "blue", "Republican": "red"},
    linewidth=2.5,
    marker="o"
)

plt.title("Tendency of incidentes by year and political party", fontsize=14)
plt.xlabel("Year")
plt.ylabel("Total number of incidents")
plt.legend(title="Political Tendency")
plt.grid(alpha=0.3)
plt.show()